In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import os,random, math, psutil, pickle

import pandas_profiling as pdp

from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

import category_encoders as ce


In [2]:
train = pd.read_csv('./input/bank_marketing_train.csv')

In [3]:
train.shape

(33744, 20)

In [4]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [5]:
train.head()

,age,job,marital,education,default,housing,loan,contact,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,56,services,married,high.school,no,no,yes,telephone,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,45,services,married,basic.9y,unknown,no,no,telephone,mon,198,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,59,admin.,married,professional.course,no,no,no,telephone,mon,139,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [6]:
train.job.value_counts()

admin.           8536
blue-collar      7591
technician       5529
services         3255
management       2359
retired          1432
entrepreneur     1203
self-employed    1130
housemaid         882
unemployed        822
student           735
unknown           270
Name: job, dtype: int64

In [7]:
train.education.value_counts()

university.degree      9971
high.school            7830
basic.9y               4992
professional.course    4265
basic.4y               3428
basic.6y               1819
unknown                1424
illiterate               15
Name: education, dtype: int64

In [8]:
education_rank = {
    "unknown": 0,
    "illiterate": 1,
    "basic.4y" : 2,
    "basic.6y"  :3,
    "basic.9y":4,
    "high.school":5, 
    "professional.course":6 ,
    "university.degree":7
}

In [9]:
train["nr.employed"].value_counts()

5228.1    13276
5099.1     6987
5191.0     6389
5195.8     3014
5076.2     1362
5017.5      880
4991.6      633
5008.7      528
4963.6      524
5023.5      143
5176.3        8
Name: nr.employed, dtype: int64

In [10]:
#pdp.ProfileReport(train)

In [11]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y'],
      dtype='object')

# 前処理

In [12]:
# durationをdropする
numeric = [ 'cons.price.idx', 'cons.conf.idx', 'age','campaign','pdays', 'previous'  ]
categoricals = ['job', 'marital',  'default', 'housing', 'loan',
#       'contact', 'day_of_week',  'poutcome', 'emp.var.rate', 'euribor3m', 'nr.employed']
     # 'contact', 'day_of_week',  'poutcome', 'euribor3m']
      'education','contact', 'day_of_week',  'poutcome', 'euribor3m']


#rank_categoricals = []
rank_categoricals = ["education"]

feat_cols = numeric + categoricals


In [13]:
def preprocessing ( df ) :
    df.drop(['duration'], axis=1, inplace=True)
    df["y"].replace("no",0, inplace=True)
    df["y"].replace("yes",1, inplace=True)
    ce_oe = ce.OrdinalEncoder(cols=categoricals,handle_unknown='impute')
    df = ce_oe.fit_transform(df)
#    df["education"] = df["education"].map( lambda x: x+"hoge" )
    #df["education"] = df["education"].map( lambda x: education_rank[x] )

    #oe = preprocessing.OrdinalEncoder()
#     encoded = oe.fit_transform(df[categoricals].values)
#     le = LabelEncoder()
#     df[[categoricals]] = le.fit_transform(df[[categoricals]])
    return df


In [14]:
train = preprocessing(train)

In [15]:
train

,age,job,marital,education,default,housing,loan,contact,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,1,1,1,1,1,1,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0
1,57,2,1,2,2,1,1,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0
2,56,2,1,2,1,1,2,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0
3,45,2,1,3,2,1,1,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0
4,59,3,1,4,1,1,1,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0
5,41,4,1,5,2,1,1,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0
6,24,5,2,4,1,2,1,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0
7,25,2,2,2,1,2,1,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0
8,29,4,2,2,1,1,2,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0
9,57,1,3,1,1,2,1,1,1,1,999,0,1,1.1,93.994,-36.4,1,5191.0,0


# モデル作成

In [16]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y'],
      dtype='object')

# ポイント

In [17]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'rmse'},
            'subsample': 0.8, # 汎化性能
            'subsample_freq': 1,
            'learning_rate': 0.01, # 0.1で試して、0.01で最終
            'num_leaves': 28, # 2^3 ~ 2^6
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'min_data_in_leaf': 5, # 汎化性能
            #'label_gain':[1,10]
            'scale_pos_weight':9 # 重みづけ
            #'device':'gpu'
            }

# folds = 5
# seed = 666
# kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

folds = 5
seed = 666

kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
#kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

In [18]:
#(train_index, val_index) = kf.split(train, train["y"])
# hoge = kf.split(train, train["y"])

i =0
for train_index, val_index in kf.split(train, train['y']):
    train_X = train[feat_cols].iloc[train_index] 
    train_y = train["y"].iloc[train_index] 
    val_X = train[feat_cols].iloc[val_index]
    val_y = train["y"].iloc[val_index]
    i = i+1
    print(1)
    break



1


In [19]:
train_X

#train_X_new = pd.DataFrame(train_X, index=train_X_origin.index , columns=train_X_origin.columns)

,cons.price.idx,cons.conf.idx,age,campaign,pdays,previous,job,marital,default,housing,loan,education,contact,day_of_week,poutcome,euribor3m
0,93.994,-36.4,56,1,999,0,1,1,1,1,1,1,1,1,1,1
1,93.994,-36.4,57,1,999,0,2,1,2,1,1,2,1,1,1,1
2,93.994,-36.4,56,1,999,0,2,1,1,1,2,2,1,1,1,1
3,93.994,-36.4,45,1,999,0,2,1,2,1,1,3,1,1,1,1
5,93.994,-36.4,41,1,999,0,4,1,2,1,1,5,1,1,1,1
6,93.994,-36.4,24,1,999,0,5,2,1,2,1,4,1,1,1,1
7,93.994,-36.4,25,1,999,0,2,2,1,2,1,2,1,1,1,1
8,93.994,-36.4,29,1,999,0,4,2,1,1,2,2,1,1,1,1
9,93.994,-36.4,57,1,999,0,1,3,1,2,1,1,1,1,1,1
10,93.994,-36.4,35,1,999,0,4,1,1,2,1,6,1,1,1,1


# ポイント

In [20]:
from imblearn.over_sampling import SMOTE

train_X_origin = train_X.copy()
train_y_origin = train_y.copy()

sm = SMOTE(random_state=42)
train_X, train_y = sm.fit_sample(train_X, train_y)

p = list(zip(train_X, train_y))
random.shuffle(p)
train_X, train_y = zip(*p)


In [21]:
train_X = pd.DataFrame(train_X, columns=train_X_origin.columns)
train_y = np.array(train_y)

In [22]:
lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)




In [23]:
train_y.sum()
train_y.shape

pd.DataFrame(train_y).tail(10)

,0
47822,0
47823,1
47824,0
47825,0
47826,1
47827,1
47828,0
47829,1
47830,1
47831,1


In [26]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=2000,
                verbose_eval = 500)
#               feval = exp_val_func)

Training until validation scores don't improve for 2000 rounds.
[500]	training's rmse: 0.42446	valid_1's rmse: 0.563275
[1000]	training's rmse: 0.375311	valid_1's rmse: 0.50787
[1500]	training's rmse: 0.349385	valid_1's rmse: 0.481717
[2000]	training's rmse: 0.33067	valid_1's rmse: 0.464356
[2500]	training's rmse: 0.314889	valid_1's rmse: 0.450651
[3000]	training's rmse: 0.301565	valid_1's rmse: 0.439531
[3500]	training's rmse: 0.289398	valid_1's rmse: 0.429764
[4000]	training's rmse: 0.27842	valid_1's rmse: 0.421318
[4500]	training's rmse: 0.26845	valid_1's rmse: 0.414122
[5000]	training's rmse: 0.259594	valid_1's rmse: 0.408126
[5500]	training's rmse: 0.251248	valid_1's rmse: 0.402595
[6000]	training's rmse: 0.243612	valid_1's rmse: 0.397584
[6500]	training's rmse: 0.236172	valid_1's rmse: 0.392923
[7000]	training's rmse: 0.229218	valid_1's rmse: 0.388726
[7500]	training's rmse: 0.222741	valid_1's rmse: 0.384935
[8000]	training's rmse: 0.216617	valid_1's rmse: 0.381375
[8500]	trainin

In [27]:
fi = pd.DataFrame(  {0: train_X.columns ,  1: gbm.feature_importance()})
fi

,0,1
0,cons.price.idx,46768
1,cons.conf.idx,42156
2,age,276872
3,campaign,115954
4,pdays,12369
5,previous,16794
6,job,86519
7,marital,53017
8,default,24883
9,housing,42786


In [28]:
from sklearn.metrics import confusion_matrix
# y_true = [0, 0, 0, 0, 1, 1, 1, 0, 1, 0]   # 実際の値 (0:スパムでない, 1:スパム)
# y_pred = [0, 0, 0, 0, 1, 1, 1, 1, 0, 1]   # 識別結果 (0:スパムでない, 1:スパム)
y_pred = gbm.predict(val_X)


In [29]:
y_pred.tolist()

[0.05801649074881218,
 0.0190480305582358,
 0.000665165932115997,
 0.00319435306538491,
 0.0015086229685203382,
 0.12072840834393683,
 0.028612392723700593,
 0.0137080503085732,
 0.0019864054339792797,
 0.01862807446226717,
 0.03927850804056302,
 0.2558221077780981,
 0.003831296563266744,
 0.0057113062190981105,
 0.13608595293933096,
 0.0017355889168616902,
 0.9139156072517166,
 0.18365054569796704,
 3.0488910146857464e-05,
 0.0023544665519464702,
 0.00010694108291861023,
 0.003490543523077714,
 0.0058074172978788636,
 0.004779451212600157,
 0.00457026271934594,
 0.07243867981108096,
 0.0038506987374446653,
 0.0062210283750517605,
 0.006443797253150884,
 0.009084777436390771,
 0.0012376270123528159,
 0.0002831922622250451,
 0.004794553414342282,
 0.016641710370857137,
 0.03512596597639134,
 0.13393682091219347,
 0.2704435801793345,
 0.008071547900041812,
 0.0011200475104872665,
 0.00043230704315082584,
 0.06251364693683487,
 0.018109543231295062,
 0.2632192224469889,
 0.006010294149854

In [30]:
val_y.tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [31]:
val_y_c = val_y.tolist()
y_pred_c =  y_pred.tolist()


# ポイント

In [32]:
# labels = ["not cvr", "cvr"]
#confusion_matrix( val_y_c, y_pred.round(), labels=labels, index=labels)

from pycm import ConfusionMatrix
cm = ConfusionMatrix(actual_vector=val_y_c, predict_vector=y_pred.round(0).astype("u1") )  
cm.print_matrix()

Predict    0          1          
Actual
0          5507       525        

1          390        327        




In [34]:
test = pd.read_csv('./input/bank_marketing_test.csv')

In [35]:
test = preprocessing(test)

In [38]:
test_X = test[feat_cols]
test_X

,cons.price.idx,cons.conf.idx,age,campaign,pdays,previous,job,marital,default,housing,loan,education,contact,day_of_week,poutcome,euribor3m
0,93.125,-35.4,37,1,999,0,1,1,1,1,1,1,1,1,1,1
1,93.125,-35.4,40,1,999,0,2,1,1,2,1,2,1,1,1,1
2,93.125,-35.4,25,1,999,0,1,2,1,1,1,1,1,1,1,1
3,93.125,-35.4,41,1,999,0,3,1,2,2,1,3,1,1,1,1
4,93.125,-35.4,37,1,999,0,2,1,1,1,1,1,1,1,1,1
5,93.125,-35.4,59,1,999,0,4,1,1,1,1,3,1,1,1,1
6,93.125,-35.4,38,1,999,0,2,2,1,2,1,4,1,1,1,1
7,93.125,-35.4,42,1,999,0,4,2,2,2,1,4,1,1,1,1
8,93.125,-35.4,39,1,999,0,5,1,1,2,2,5,1,1,1,1
9,93.125,-35.4,42,1,999,0,3,1,2,1,1,2,1,1,1,1


In [36]:

test_y = np.array(test["y"])
test_y

array([0, 0, 0, ..., 0, 0, 0])

In [39]:
y_test_pred = gbm.predict(test_X)

In [40]:
test_y_c = test_y.tolist()
y_test_pred_c =  y_test_pred.tolist()

In [41]:
from pycm import ConfusionMatrix
cm_t = ConfusionMatrix(actual_vector=test_y_c, predict_vector=y_test_pred.round(0).astype("u1") )  
cm_t.print_matrix()

Predict    0          1          
Actual
0          6980       320        

1          839        98         




# 何がダメだったか？

→もっと電話かけた方が良い！！
→多分、なんらか環境要因的にテストはトレインより低かった